In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import seaborn
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from pycaret.classification import *
from pycaret.classification import evaluate_model
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import selenium
from selenium import webdriver
import random
from selenium.webdriver.common.by import By
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
pd.set_option('display.max_columns', 100)

In [3]:
def kelly(p, b, weight):
    stake = round((p * (b - 1) - (1 - p)) / (b - 1), 4) * weight
    return stake if stake > 0.0 else 'no value'

In [104]:
setup_df = pd.read_csv('f_eng_df.csv', index_col = [0])
setup_df = setup_df[setup_df['odds_sum'] > 1.03]
setup_df.shape

(358, 106)

In [131]:
setup_df.columns[-30:]

Index(['aerial_duels_won_percentage_diff', 'aerial_duels_won_percentage_sum',
       'successful_dribbles_diff', 'successful_dribbles_sum',
       'successful_dribbles_percentage_diff',
       'successful_dribbles_percentage_sum', 'score_diff', 'score_sum',
       'posession_diff', 'posession_sum', 'odds_sum', 'xg_penalty_diff',
       'xg_penalty_sum', 'odds_predict', 'std_0', 'std_1', 'std_2', 'med_0',
       'med_1', 'med_2', '75_0', '75_1', '75_2', 'med_odds_prob_0',
       'med_odds_prob_1', 'med_odds_prob_2', '75_odds_prob_0',
       '75_odds_prob_1', '75_odds_prob_2', 'target'],
      dtype='object')

In [143]:
grid = setup(data = setup_df, target='target', session_id = 233,
             remove_perfect_collinearity = False, feature_ratio = True)

,Description,Value
0,session_id,233
1,Target,target
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(358, 106)"
5,Missing Values,True
6,Numeric Features,103
7,Categorical Features,2
8,Ordinal Features,False
9,High Cardinality Features,False


In [146]:
pd.concat([get_config('X'), get_config('y')], axis = 1).to_csv('for_tf_df.csv')

In [148]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7560,0.9140,0.7371,0.7642,0.7493,0.6212,0.6279,0.1750
xgboost,Extreme Gradient Boosting,0.7320,0.8927,0.7094,0.7404,0.7253,0.5832,0.5917,0.3350
gbc,Gradient Boosting Classifier,0.7120,0.8853,0.6729,0.7161,0.6994,0.5470,0.5581,1.0240
rf,Random Forest Classifier,0.6320,0.8241,0.5844,0.6028,0.6028,0.4209,0.4348,0.2020
et,Extra Trees Classifier,0.6160,0.8132,0.5644,0.5990,0.5922,0.3933,0.4068,0.1940
ada,Ada Boost Classifier,0.6120,0.7747,0.5895,0.6242,0.6070,0.4001,0.4072,0.0940
nb,Naive Bayes,0.6080,0.7901,0.5605,0.5724,0.5741,0.3842,0.4029,0.0090
dt,Decision Tree Classifier,0.5600,0.6661,0.5360,0.5721,0.5541,0.3247,0.3305,0.0140
svm,SVM - Linear Kernel,0.5600,0.0000,0.5070,0.5500,0.5207,0.2980,0.3248,0.0320
knn,K Neighbors Classifier,0.5240,0.7056,0.5052,0.5618,0.5333,0.2795,0.2848,0.0530


LGBMClassifier(bagging_fraction=0.8, bagging_freq=0, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.5,
               importance_type='split', learning_rate=0.001, max_depth=-1,
               min_child_samples=31, min_child_weight=0.001, min_split_gain=0,
               n_estimators=220, n_jobs=-1, num_leaves=100, objective=None,
               random_state=233, reg_alpha=0.15, reg_lambda=1e-07,
               silent='warn', subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)


In [153]:
xgb = create_model('xgboost')
tuned = tune_model(xgb, optimize = 'recall')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7200,0.8838,0.6919,0.7196,0.7094,0.5547,0.5654
1,0.6800,0.8462,0.6452,0.6640,0.6649,0.5000,0.5052
2,0.6800,0.8558,0.6705,0.6760,0.6731,0.5062,0.5100
3,0.6800,0.8641,0.6591,0.7349,0.6946,0.5181,0.5270
4,0.6800,0.8386,0.6225,0.7333,0.6585,0.4792,0.5024
5,0.8000,0.8624,0.7881,0.8016,0.7989,0.6951,0.6968
6,0.7600,0.9070,0.7296,0.7927,0.7528,0.6231,0.6349
7,0.7200,0.9110,0.6889,0.7200,0.7113,0.5625,0.5684
8,0.8400,0.9498,0.8185,0.8600,0.8393,0.7500,0.7579


In [154]:
evaluate_model(tuned)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [155]:
predicted_df = predict_model(best, raw_score = True)
# predicted_df = pd.concat([get_config('X_test').reset_index(drop = True), predicted_df], axis = 1)
# bo_po_dha = list(zip(predicted_df[['med_0','med_1','med_2']],predicted_df[['Score_0','Score_1','Score_2']],['draw','home','away']))
bo_po_dha = list(zip(predicted_df[['75_0','75_1','75_2']],predicted_df[['Score_0','Score_1','Score_2']],['draw','home','away']))
for book_odds, pred_odds, dha in bo_po_dha:
    predicted_df[dha] = predicted_df.apply(lambda x: kelly(x[pred_odds], x[book_odds], 0.25), axis=1)
df = predicted_df
dha = {'draw':'0','home':'1','away':'2'}
df = df.replace('no value', 0)
df[['draw','home','away']] = df[['draw','home','away']].astype(float)
for word, number in dha.items():
    df[f'{word}_profits'] = np.where(df['target'] == int(number),
                                    df[word] * (df['75_'+ number] - 1) * 100,
                                    -100*df[word])
df['profits'] = df[df.columns[-3:]].sum(axis = 1)
p = round(np.sum(df["profits"]), 3)
df = df.round(3)
df.to_csv(f'profits_{datetime.now():%Y%m%d}.csv')
print(f'number of bets made: {df.shape[0]}')
print(f'profit on $100 bankroll: ${p}')
print(df.sum()[-4:-1])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.5741,0.7573,0.5314,0.5730,0.5733,0.3387,0.3388


number of bets made: 108
profit on $100 bankroll: $-164.291
draw_profits   -99.444
home_profits     5.828
away_profits   -70.675
dtype: float64


In [157]:
df[['odds_sum','score_diff','target', 'Label','75_0', '75_1', '75_2', 'Score_0', 'Score_1', 'Score_2', 'draw', 'home',
       'away', 'draw_profits', 'home_profits', 'away_profits', 'profits']].head(20)

,odds_sum,score_diff,target,Label,75_0,75_1,75_2,Score_0,Score_1,Score_2,draw,home,away,draw_profits,home_profits,away_profits,profits
0,1.054,1.0,1,1,17.000,1.040,71.000,0.008,0.991,0.001,0.000,0.191,0.000,-0.000,0.763,-0.000,0.763
1,1.098,-1.0,2,2,3.550,6.500,1.600,0.047,0.000,0.953,0.000,0.000,0.219,-0.000,-0.000,13.128,13.128
2,1.073,0.0,0,1,4.962,1.250,17.000,0.016,0.982,0.001,0.000,0.228,0.000,0.000,-22.787,-0.000,-22.787
3,1.079,-1.0,2,2,4.300,11.250,1.356,0.024,0.000,0.975,0.000,0.000,0.227,-0.000,-0.000,8.056,8.056
4,1.081,0.0,1,1,2.550,3.477,2.700,0.030,0.693,0.278,0.000,0.142,0.000,-0.000,35.208,-0.000,35.208
5,1.076,1.0,0,1,3.500,1.700,5.500,0.031,0.885,0.085,0.000,0.180,0.000,0.000,-17.992,-0.000,-17.992
6,1.084,0.0,1,1,3.000,1.667,8.000,0.020,0.977,0.003,0.000,0.236,0.000,-0.000,15.712,-0.000,15.712
7,1.072,0.0,1,0,3.400,1.533,9.000,0.624,0.376,0.000,0.117,0.000,0.000,-11.665,0.000,-0.000,-11.665
8,1.075,-1.0,2,2,4.750,13.000,1.293,0.004,0.002,0.994,0.000,0.000,0.244,-0.000,-0.000,7.134,7.134
9,1.073,0.0,0,2,3.511,8.000,1.571,0.000,0.000,0.999,0.000,0.000,0.249,0.000,-0.000,-24.930,-24.930
